In [1]:
import urllib
import re
from bs4 import BeautifulSoup
import csv
from geopy.geocoders import Nominatim
import pandas as pd
import numpy

In [2]:
def getLinks(html, maxlinks=40):
    url = []
    cursor = 0
    nlinks = 0
    while(cursor>=0 and maxlinks>nlinks):
        start_link = html.find('a href', cursor)
        if start_link == -1:
            return url
        start_quote = html.find('"',start_link)
        end_quote = html.find('"',start_quote+1)
        url.append(html[start_quote+1:end_quote])
        cursor = end_quote+1
        nlinks=nlinks+1
    return url

In [3]:
url = "http://barcelonaapi.marcpous.com/"
raw = urllib.request.urlopen(url).read()
linkis = getLinks(str(raw))

flinks = []
for l in linkis:
    if bool(re.match(r'.*stations$',l)):
        flinks.append(l)

In [4]:
flinks

['/bicing/stations',
 '/bus/stations',
 '/metro/stations',
 '/tram/stations',
 '/fgc/stations',
 '/renfe/stations']

In [5]:
for i in range(len(flinks)):
    name = flinks[i].split('/')[1]
    with open(name + '.csv', 'w', newline = '\n', encoding = 'UTF-8') as csvfile:
        out =csv.writer(csvfile, delimiter = ',')
        text = urllib.request.urlopen(url +flinks[i]).read()
        soup = BeautifulSoup(text,'lxml')
        l = []
        counter = 0
        for tr in soup.find_all('tr'):
            tds = tr.find_all('td')
            lk = []
            for i in range(len(tds)):
                lk.append(tds[i].text) 
            l.append(lk)
        out.writerows(l)

In [6]:
renfe = pd.read_csv('renfe.csv',header=0)
tram =  pd.read_csv('tram.csv',header=0)
fgc = pd.read_csv('fgc.csv',header=0)
bicing = pd.read_csv('bicing.csv', header=0)
metro = pd.read_csv('metro.csv', header = 0)
bus = pd.read_csv('bus.csv', header = 0)

In [7]:
bicing.rename(columns={'Lat / Lon':'Lat/Lon'}, inplace=True)

In [8]:
def locsep(frame):
    frame['Lat/Lon'] = [frame['Lat/Lon'][i].replace('/',',') for i in range(len(frame['Lat/Lon']))]
    return frame

In [9]:
tram = locsep(tram)
metro =locsep(metro)
fgc = locsep(fgc)
bicing  = locsep(bicing)
renfe = locsep(renfe)
bus = locsep(bus)

,ID,Name,Line,Lat/Lon,Operator,Type,Zone
0,216,Ulldecona-Alcanar-La Sènia,R16,"40.5958608323863 , 0.449509090610943",RENFE,REGIONALS,NaN
1,215,Tortosa,R16,"40.8071840875766 , 0.522786433876677",RENFE,REGIONALS,NaN
2,214,Camp-redó,R16,"40.7590269187727 , 0.556463577981504",RENFE,REGIONALS,NaN
3,213,L'Aldea-Amposta,R16,"40.7537372646208 , 0.614209535774528",RENFE,REGIONALS,NaN
4,211,Camarles-Deltebre,R16,"40.7751518042299 , 0.671158009592543",RENFE,REGIONALS,NaN
5,212,L'Ampolla-Perelló-Deltebre,R16,"40.8121075583674 , 0.708625792133952",RENFE,REGIONALS,NaN
6,210,L'Ametlla de Mar,R16,"40.8863485439212 , 0.800863850578691",RENFE,REGIONALS,NaN
7,209,L'Hospitalet de l'Infant,R16,"40.9959991362972 , 0.920691759813987",RENFE,REGIONALS,NaN
8,208,Mont-roig del Camp,R16,"41.0448814724457 , 0.990822798035191",RENFE,REGIONALS,NaN
9,205,Riba-roja d'Ebre,R15,"41.2486583699538 , 0.480955059648806",RENFE,REGIONALS,NaN


In [17]:
def neighdistr(frame):
    geolocator = Nominatim(timeout=100)
    frame['Districte'] = ""
    frame['Barri'] = ""
    for i in range(len(frame['Lat/Lon'])):
        location = geolocator.reverse(frame['Lat/Lon'][i])
        if 'city' in location.raw['address']:
            if (location.raw['address']['city'] == 'Barcelona'):
                if 'city_district' in location.raw['address']:
                    frame.loc[i,'Districte']  = location.raw['address']['city_district']
                if 'neighbourhood' in location.raw['address']:    
                    frame.loc[i,'Barri'] = location.raw['address']['neighbourhood']


In [14]:
neighdistr(bus)

GeocoderServiceError: HTTP Error 429: Too Many Requests

In [20]:
#neighdistr(renfe)
##neighdistr(bicing)
#neighdistr(fgc)
#neighdistr(metro)
neighdistr(bus)

GeocoderServiceError: Remote end closed connection without response

In [13]:
renfe.to_csv('renfe.csv', sep='\t', encoding='utf-8')
tram.to_csv('tram.csv', sep='\t', encoding='utf-8')
fgc.to_csv('fgc.csv', sep='\t', encoding='utf-8')
#bus.to_csv('bus.csv', sep='\t', encoding='utf-8')
metro.to_csv('metro.csv', sep='\t', encoding='utf-8')
bicing.to_csv('bicing.csv', sep='\t', encoding='utf-8')
